In [1]:
from torchvision.datasets import ImageFolder

## Import dataset
dataset = ImageFolder("data/plantvillage dataset/combined")

## Display dataset classes and the length of the dataset
print(dataset.classes)
print(len(dataset))

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

In [2]:
from torchvision import transforms

## Transforms the data that is used for training
train_transform = transforms.Compose([
    ## Resizes the image
    transforms.Resize(256),
    ## Randomly crops the image in order to increase the number of images that are used to train the model.
    transforms.RandomCrop(224),
    ## Randomly flips images horizontally in order to increase the number of images that are used to train the model.
    transforms.RandomHorizontalFlip(),
    ## Transforms the images used for training into tensors
    transforms.ToTensor(),
    ## Normalizes the RGB values based on the ImageNet means and standard deviations
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Transforms the data that is used for training
val_transform = transforms.Compose([
    ## Resizes the image
    transforms.Resize(256),
    ## Center crops the images that are used for validation
    transforms.CenterCrop(224),
    ## Transforms the images used to validation into tensors
    transforms.ToTensor(),
    ## Normalizes the RGB values based on the ImageNet means and standard deviations
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## The test transforms are the same as the validation transforms.
test_transform = val_transform

In [3]:
from torch.utils.data import random_split

## Creates the training, validation, and testing datasets
train_dataset = ImageFolder("data/plantvillage dataset/combined", transform=train_transform)
val_dataset = ImageFolder("data/plantvillage dataset/combined", transform=val_transform)
test_dataset = ImageFolder("data/plantvillage dataset/combined", transform=test_transform)

## identify training, validation, and test sizes
train_size = int(0.7 * len(train_dataset))
val_size = int(0.15 * len(train_dataset))
test_size = len(train_dataset) - train_size - val_size

In [4]:
from torch.utils.data import Subset
import torch

## This is the seed that is used when training the model. We use this in order to get the same results across training sessions.
torch.manual_seed(42)

## This creates a random permutation of the indices based on the size indicated.
indices = torch.randperm(len(train_dataset)).tolist()
## This identifies the indices that are used for the training dataset
train_indices = indices[:train_size]
## This identifies the indices that are used for the validation dataset
val_indices = indices[train_size:train_size+val_size]
## This identifies the indices that are used for the testing dataset
test_indices = indices[train_size+val_size:]

## These create new training, validation, and testing datasets that are the sizes indicated by their corresponding indices
train_dataset = Subset(train_dataset, train_indices)
val_dataset = Subset(val_dataset, val_indices)
test_dataset = Subset(test_dataset, test_indices)

In [5]:
from torch.utils.data import DataLoader

## These create the loaders for the training, validation, and testing datasets
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

In [6]:
images, labels = next(iter(train_loader))
print(f"Batch shape: {images.shape}")  # Should be [32, 3, 224, 224]
print(f"Image value range: {images.min():.3f} to {images.max():.3f}")  # Should be around -2 to 2 (normalized)
print(f"Labels shape: {labels.shape}")  # Should be [32]

Batch shape: torch.Size([32, 3, 224, 224])
Image value range: -2.118 to 2.640
Labels shape: torch.Size([32])


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

using device: cuda


In [8]:
from torchvision import models
import torch.nn as nn

## This assigns the model to the resnet18 pretrained model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

## The number of features is assigned to the number of in_features.
num_ftrs = model.fc.in_features
## The fully connected layer is assigned Linear module in the neural network module of torch. This makes the output size 38.
model.fc = nn.Linear(num_ftrs, 38)
print(f"model: {model}")
print(f"model.fc: {model.fc}")

## This moves the model to the device (CPU)
model = model.to(device)

model: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=T

In [9]:
import torch.optim as optim

## This identifies the loss function that is being used
criterion = nn.CrossEntropyLoss()

## This identifies the optimizer that is used for the model.
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
from tqdm import tqdm

## This identifies the number of epochs
num_epochs = 10

## This loop trains the model
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 30)

    model.train()

    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for images, labels in tqdm(train_loader, desc="Training"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = 100 * train_correct / train_total

    model.eval()

    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  ## Disable gradient calculation
        for images, labels in tqdm(val_loader, desc="Validation"):
            ## Move data to CPU
            images = images.to(device)
            labels = labels.to(device)

            ## Forward pass only (no backward pass needed)
            outputs = model(images)
            loss = criterion(outputs, labels)

            ## Track statistics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    ## Calculate epoch statistics
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total

    ## -------------------- PRINT RESULTS --------------------
    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.2f}%")
    print(f"Val Loss:   {avg_val_loss:.4f} | Val Acc:   {val_accuracy:.2f}%")

print("\nTraining complete!")


Epoch 1/10
------------------------------


Validation: 100%|██████████| 510/510 [00:20<00:00, 24.49it/s]


Train Loss: 0.3008 | Train Acc: 90.68%
Val Loss:   0.1364 | Val Acc:   95.46%

Epoch 2/10
------------------------------


Validation: 100%|██████████| 510/510 [00:20<00:00, 24.81it/s]


Train Loss: 0.1405 | Train Acc: 95.48%
Val Loss:   0.0982 | Val Acc:   96.86%

Epoch 3/10
------------------------------


...

Train Loss: 0.0502 | Train Acc: 98.36%
Val Loss:   0.0400 | Val Acc:   98.67%

Epoch 9/10
------------------------------


Validation: 100%|██████████| 510/510 [00:21<00:00, 23.69it/s]


Train Loss: 0.0445 | Train Acc: 98.61%
Val Loss:   0.0326 | Val Acc:   98.90%

Epoch 10/10
------------------------------


Validation: 100%|██████████| 510/510 [00:20<00:00, 24.58it/s]

Train Loss: 0.0400 | Train Acc: 98.71%
Val Loss:   0.0547 | Val Acc:   98.43%

Training complete!


In [11]:
torch.save(model.state_dict(), "plant_disease_model.pth")
print("model saved")

model saved


In [12]:
model.eval()

test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct +=  (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f"Final Test Accuracy: {test_accuracy:.2f}%")
print(f"Correctly classified {test_correct} out of {test_total} images")

Testing: 100%|██████████| 510/510 [00:21<00:00, 23.99it/s]

Final Test Accuracy: 98.64%
Correctly classified 16071 out of 16293 images


In [13]:
from PIL import Image

def predict_image(image_path, model, transform, class_names, device):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0)
    image_tensor = image_tensor.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        confidence, predicted_idx = torch.max(probabilities, 1)

    predicted_class = class_names[predicted_idx.item()]
    confidence_percent = confidence.item() * 100

    return predicted_class, confidence_percent

In [14]:
prediction, confidence = predict_image("./data/plantvillage dataset/combined/Tomato___Early_blight/004cbe60-8ff9-4965-92df-e86694d5e9ba___RS_Erly.B 8253_final_masked.jpg", model, val_transform, dataset.classes, device)

print(f"Predicted: {prediction}")
print(f"Confidence: {confidence:.2f}%")

Predicted: Tomato___Early_blight
Confidence: 99.89%
